In [1]:
import torch
import numpy as np

total_sample = 500
total_time_steps = 100
n_points = 50

# 设置训练集和测试集的边界
boundary = int(total_sample*4/5)

batch_size = 10

hidden_dims = [200,200,200,200]
output_dim = 50

epochs = 2

In [2]:
# In this cell, we define the function to get the cell centers of a 1D mesh. 
# Also, we set up the spatial and temporal grid points for the training and testing datasets.
# This is the so-called y_expanded tensor. 
import fipy as fp

def get_cell_centers(n_points = 50):
    """
    Get the cell center positions for a 1D mesh with the specified number of grid points.

    Parameters:
    - n_points: Number of grid points in the spatial domain.

    Returns:
    - cell_centers: The x-positions of the cell centers.
    """
    L = 1.0  # Length of the domain
    dx = L / n_points

    # Create a 1D mesh
    mesh = fp.Grid1D(nx=n_points, dx=dx)

    # Get the cell center positions
    cell_centers = mesh.cellCenters[0]  # These are the x-positions of the cell centers
    cell_centers = np.array(cell_centers)

    return cell_centers

# Example usage:
cell_centers = get_cell_centers(n_points)
cell_centers = np.around(cell_centers, decimals=2)
time_steps = np.linspace(0.01, 1, total_time_steps)
time_steps = np.around(time_steps, decimals=2)

#print(time_steps)

Y1, Y2 = np.meshgrid(cell_centers, time_steps)  # 第一个变量进行行展开，第二个变量进行列展开

y = np.column_stack([Y2.ravel(),Y1.ravel()]) 
# 先将 Y2 和 Y1 进行展开，然后将展开后的两个向量进行列合并

y_tensor = torch.tensor(y, dtype=torch.float64)
print(f"The dimension of y_tensor is {y_tensor.shape}.")
y_expanded = y_tensor.unsqueeze(0).expand(total_sample, -1, -1)
print(f"The dimension of y_expanded is {y_expanded.shape} after expanding.")

The dimension of y_tensor is torch.Size([5000, 2]).
The dimension of y_expanded is torch.Size([500, 5000, 2]) after expanding.


In [3]:
# In this cell, we load the initial conditions and solutions from the saved files.
import os
import numpy as np

# Define the directory where you want to save the file
data_directory = os.path.join(os.getcwd(), 'data')

initials_name = f'heat_initials_{len(cell_centers)}.npy'
solutions_name = f'heat_solutions_{len(cell_centers)}.npy'

# Define the file paths
initials_path = os.path.join(data_directory, initials_name)
solutions_path = os.path.join(data_directory, solutions_name)

# Load the data
initials = np.load(initials_path)
solutions = np.load(solutions_path)

print(f"The dimensions of the initial conditions are: {initials.shape}")
print(f"The dimensions of the solutions are: {solutions.shape}")

The dimensions of the initial conditions are: (500, 50)
The dimensions of the solutions are: (500, 100, 50)


In [4]:
# In this cell, we arrange the initial conditions into the desired format for training the DeepONet.
# This is the so-called u_expanded tensor.

u_tensor = torch.tensor(initials, dtype=torch.float64)
print(f"The dimension of u_tensor is {u_tensor.shape}.")

u_expanded = u_tensor.unsqueeze(1) # u_expanded: tensor[total_sample, 1, n_points]
u_expanded = u_expanded.expand(-1, total_time_steps*n_points, -1) # u_expanded: tensor[total_sample, total_time_steps*n_points, n_points]
print(f"The dimension of u_expanded is {u_expanded.shape} after expanding.")

The dimension of u_tensor is torch.Size([500, 50]).
The dimension of u_expanded is torch.Size([500, 5000, 50]) after expanding.


In [5]:
# In this cell, we arrange the solutions into the desired format for training the DeepONet.
# This is the so-called s_expanded tensor.

solutions_linear = np.zeros((total_sample, total_time_steps*n_points))

for i in range(total_sample):
    solutions_linear[i] = solutions[i].flatten()

# solutions is a 3D array of shape (total_sample, total_time_steps, n_points)
print(f"The loaded solution dataset has dimension {solutions.shape},\n\t while the arranged linearized dataset has dimension {solutions_linear.shape}.")

s_tensor  = torch.tensor(solutions_linear, dtype=torch.float64) # s_tensor: tensor[total_sample, total_time_steps*n_points]
s_expanded  = s_tensor.unsqueeze(2) # s_expanded: tensor[total_sample, total_time_steps*n_points, 1]

print(f"The dimension of s_tensor is {s_tensor.shape}.")
print(f"The dimension of s_expanded is {s_expanded.shape} after expanding.")

The loaded solution dataset has dimension (500, 100, 50),
	 while the arranged linearized dataset has dimension (500, 5000).
The dimension of s_tensor is torch.Size([500, 5000]).
The dimension of s_expanded is torch.Size([500, 5000, 1]) after expanding.


In [6]:
"""
This is the function to well organize the dataset
"""
import torch
from torch.utils.data import Dataset

class CustomDataset(Dataset):
    def __init__(self, input1_data, input2_data, targets):
        self.input1_data = input1_data
        self.input2_data = input2_data
        self.targets = targets

    def __len__(self):
        return len(self.input1_data)

    def __getitem__(self, idx):
        input1 = self.input1_data[idx]
        input2 = self.input2_data[idx]
        target = self.targets[idx]
        return input1, input2, target

In [7]:
from torch.utils.data import DataLoader
train_set = CustomDataset(u_expanded[:boundary], y_expanded[:boundary], s_expanded[:boundary])
test_set = CustomDataset(u_expanded[boundary:], y_expanded[boundary:], s_expanded[boundary:])

# 创建 DataLoader
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=2) 
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False, num_workers=2) 

In [8]:
"""
Design a DeepONet.
"""
import torch
import torch.nn as nn
import torch.optim as optim

# Branch Network
class BranchNet(nn.Module):
    def __init__(self, input_dim, hidden_dims, output_dim):
        super(BranchNet, self).__init__()
        layers = []
        in_dim = input_dim
        
        # 添加多个隐藏层
        for h_dim in hidden_dims:
            layers.append(nn.Linear(in_dim, h_dim))
            layers.append(nn.ELU())
            in_dim = h_dim
        
        layers.append(nn.Linear(in_dim, output_dim))
        self.fc = nn.Sequential(*layers)
        
    def forward(self, x):
        return self.fc(x)

# Trunk Network
class TrunkNet(nn.Module):
    def __init__(self, input_dim, hidden_dims, output_dim):
        super(TrunkNet, self).__init__()
        layers = []
        in_dim = input_dim
        
        # 添加多个隐藏层
        for h_dim in hidden_dims:
            layers.append(nn.Linear(in_dim, h_dim))
            layers.append(nn.ELU())
            in_dim = h_dim
        
        layers.append(nn.Linear(in_dim, output_dim))
        self.fc = nn.Sequential(*layers)
        
    def forward(self, y):
        return self.fc(y)

# DeepONet
class DeepONet(nn.Module):
    def __init__(self, branch_input_dim, trunk_input_dim, hidden_dims, output_dim):
        super(DeepONet, self).__init__()
        self.branch_net = BranchNet(branch_input_dim, hidden_dims, output_dim)
        self.trunk_net = TrunkNet(trunk_input_dim, hidden_dims, output_dim)
        
    def forward(self, x, y):
        branch_output = self.branch_net(x)
        trunk_output = self.trunk_net(y)
        # Combine the outputs (typically element-wise product)
        output = torch.sum(branch_output * trunk_output, dim=-1, keepdim=True) # 按照最后一个坐标做内积
        return output

In [9]:
def mse(prediction, target):
    ms_loss = torch.mean((prediction - target) ** 2)
    return ms_loss

In [10]:
# Hyperparameters
branch_input_dim = n_points  # Number of points to represent the original function
trunk_input_dim = 2     # Coordinate where we evaluate the transformed function

# Create model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = DeepONet(branch_input_dim, trunk_input_dim, hidden_dims, output_dim).to(device)

In [11]:
optimizer = torch.optim.Adamax(model.parameters(), lr=0.001)

# 训练模型
error_list = []
for epoch in range(epochs):
    print(f"Epoch {epoch+1}")
    err = []
    for input1_batch, input2_batch, target_batch in train_loader:
        optimizer.zero_grad()
        outputs = model(input1_batch, input2_batch)
        loss = mse(outputs, target_batch)
        err.append(loss.item())

        loss.backward()
        optimizer.step()
        del input1_batch, input2_batch, target_batch, outputs, loss
        torch.cuda.empty_cache()  # 释放当前批次的缓存
    error_list.append(err)

Epoch 1


RuntimeError: DataLoader worker (pid(s) 24092, 21496) exited unexpectedly